# Notebook responsável por apresentar os comandos básicos para manipulação de dados no mongo

## Crud com apenas 1 documento
- Inserir 1 documento **composto**
- Recuperar 1 documento **composto**
- Atualizar 1 documento **composto**
- Excluir 1 documento **composto**

## Bibliotecas importantes

In [ ]:
from pymongo import MongoClient
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker


## Funções uteis

In [ ]:
def criar_endereco():
    return {
        'logradouro':fake.street_name(),
        'numero':fake.building_number(),
        'bairro':fake.bairro(),
        'cidade':fake.city(),
        'uf':fake.administrative_unit(),
        'cep':fake.postcode(),
    }

In [ ]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(18, 50)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
        'data_nascimento':fake.date()
    }        
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    pessoa['endereco'] = criar_endereco()

    pessoa['telefones'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['telefones'].append(fake.msisdn())

    pessoa['formacoes'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['formacoes'].append(fake.job())
        
        
    return pessoa

## Configurações

In [ ]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de 1 documento

### Conectando ao MongoDB

In [ ]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_mongo02']

### Exemplo - Criação de um documento composto

In [ ]:
pessoa = criar_pessoa()
pp.pprint(pessoa)

In [ ]:
df = pd.DataFrame.from_records([pessoa])
df

In [ ]:
df.info()

In [ ]:
df.loc[0, 'data_nascimento']

In [ ]:
df.loc[0, 'endereco']

In [ ]:
type(df.loc[0, 'endereco'])

In [ ]:
df.loc[0, 'telefones']

In [ ]:
type(df.loc[0, 'telefones'])

In [ ]:
df.loc[0, 'formacoes']

In [ ]:
type(df.loc[0, 'formacoes'])

### Inserindo o documento no banco de dados

In [ ]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_id = colecao_pessoa.insert_one(pessoa).inserted_id
pessoa_id

## Recuperando o documento inserido

#### Recuperar pelo ID

In [ ]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

#### Recuperando por algum outro atributo

In [ ]:
criterio = {
    'idade': {'$eq':pessoa['idade']}
}
pp.pprint(criterio)


In [ ]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

#### Recuperando por algum outro atributo

In [ ]:
criterio = {
    'nome': {'$eq':pessoa['nome']}
}
pp.pprint(criterio)


In [ ]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

In [ ]:
criterio = {
    'nome': {'$eq':'nome_estranho_que_não_será_criado'}
}
pp.pprint(criterio)

In [ ]:
p = colecao_pessoa.find_one(criterio)
p, type(p)

## Atualizar o documento

In [ ]:
criterio = {
    "_id":pessoa_id}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)


In [ ]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [ ]:
# ref.: https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html
r.matched_count

In [ ]:
r.modified_count

In [ ]:
r.raw_result

In [ ]:
r.upserted_id

In [ ]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

## Atualizar o documento (que não existe)

In [ ]:
criterio = {
    "_id":'123'}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)

In [ ]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [ ]:
r.matched_count

In [ ]:
r.modified_count

In [ ]:
r.raw_result

In [ ]:
r = colecao_pessoa.update_one(criterio, novo_valor, upsert=True)
r

In [ ]:
r.raw_result

In [ ]:
r.upserted_id

In [ ]:
p = colecao_pessoa.find_one({"_id":r.upserted_id})
pp.pprint(p)

## Remover o documento

In [ ]:
criterio = {
    "_id":pessoa_id}
pp.pprint(criterio)

In [ ]:
r = colecao_pessoa.delete_one(criterio)

In [ ]:
r.raw_result

In [ ]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)